In [169]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [170]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH__PROJECT")

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [171]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000262E9794680>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000262E8823E30>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [172]:
llm.invoke(input="what is your training parameter")

AIMessage(content="I can't share specific details about my training parameters. This information is typically kept confidential by my creators, the Gemma team, to protect the model and prevent misuse.\n\nHowever, I can tell you some general things about how large language models like me are trained:\n\n* **Massive Datasets:** We are trained on enormous amounts of text data, which allows us to learn patterns in language and understand how words and concepts relate to each other.\n* **Deep Learning Algorithms:** We use deep learning algorithms, specifically a type of neural network called a transformer, to process and analyze the data.\n\n* **Optimization Techniques:**\n\nOur training involves complex optimization techniques to adjust the model's parameters and improve its performance.\n\n* **Evaluation Metrics:**\n\nOur performance is evaluated using various metrics, such as accuracy, fluency, and\n\ncomprehensiveness.\n\nIf you're interested in learning more about the training of large

In [173]:
'''from langchain_core.messages import HumanMessage, AIMessage

llm.invoke([HumanMessage(content="")])


llm.invoke(
    [
        HumanMessage(content="Hi , My name is Darshan and I am a Chief AI Engineer"),
        AIMessage(content="Hello Darshan! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days?\n\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)''' 
# It uses past conversation to give next questions answer...See the above cell, we manually adede AIMessage here. What is the dinamic approach...?

'from langchain_core.messages import HumanMessage, AIMessage\n\nllm.invoke([HumanMessage(content="")])\n\n\nllm.invoke(\n    [\n        HumanMessage(content="Hi , My name is Darshan and I am a Chief AI Engineer"),\n        AIMessage(content="Hello Darshan! It\'s nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days?\n\n"),\n        HumanMessage(content="Hey What\'s my name and what do I do?")\n    ]\n)'

In [174]:
from langchain_community.chat_message_histories import ChatMessageHistory # store and manage the conversation's messages in a structured way..
from langchain_core.chat_history import BaseChatMessageHistory # adding, retrieving, or clearing messages) for any chat history implementation..
from langchain_core.runnables.history import RunnableWithMessageHistory

### ChatMessageHistory    
In memory implementation of chat message history. store and manage the conversation's messages in a structured way..


### BaseChatMessageHistory     
Abstract base class for storing chat message history.
Think of an abstract base class like a blueprint for building something. It outlines the required parts (methods and properties) that every finished product (or subclass) should have, but it doesn't build anything on its own. You must create a specific subclass that fills in the details to actually use it.

### RunnableWithMessageHistory
Runnable that manages chat message history for another Runnable.

A chat message history is a sequence of messages that represent a conversation.

RunnableWithMessageHistory wraps another Runnable and manages the chat message
history for it; it is responsible for reading and updating the chat message
history.

In [175]:
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm, get_session_history)

# This code sets up a simple session-based chat history system. 
# It uses a dictionary to store a chat history for each session. 
# The function checks if a session already has a history, and if not, creates one using the concrete class. 
# Finally, it wraps a language model with a runnable that automatically handles chat history for each session.

In [176]:
print(with_message_history)

bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]) kwargs={} config={} config_factories=[] get_session_history=<function get_session_history at 0x00000262E96EACA0> history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)]


In [177]:
config = {"configurable": {"session_id": "d9"}}
# The runnable can handle chat histories for multiple sessions, and 
# the configuration specifies which session ID to use for the current operation.
'''Note: By default, the Runnable is expected to take a single configuration parameter
called `session_id` which is a string. This parameter is used to create a new
or look up an existing chat message history that matches the given session_id.'''

'Note: By default, the Runnable is expected to take a single configuration parameter\ncalled `session_id` which is a string. This parameter is used to create a new\nor look up an existing chat message history that matches the given session_id.'

In [178]:
store

{}

In [179]:
from langchain_core.messages import HumanMessage, AIMessage
response = with_message_history.invoke([HumanMessage(content="Hi, I am a darshan, a data scientist.")], config=config)
response

AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80})

In [180]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80})])}

In [181]:
with_message_history.invoke([HumanMessage(content="I am working on Agentic AI.")], config=config)

AIMessage(content="That's fascinating! Agentic AI is a really exciting field. \n\nCould you tell me a bit more about what you're working on specifically? For example:\n\n* **What kind of agents are you developing?** (e.g., software agents, embodied agents, multi-agent systems)\n* **What tasks are these agents designed to perform?** (e.g., planning, decision-making, interacting with the environment)\n* **What are some of the challenges you're facing in your work?**\n\nI'm eager to learn more about your project!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 96, 'total_tokens': 221, 'completion_time': 0.227272727, 'prompt_time': 0.005262967, 'queue_time': 0.11001534599999999, 'total_time': 0.232535694}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-81f1f30c-f6d6-469a-9d85-cb878beb1c6a-0', usage_metadata={'input_tokens': 96, 'output_tokens': 125, 'total_token

In [182]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}), HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}), AIMessage(content="Tha

Now see the difference b/w two session Id..

In [183]:
# first example: this session Id we have created and have already given the information...d9
config = {"configurable": {"session_id": "d9"}} 
with_message_history.invoke([HumanMessage(content="who Am I?")], config=config)

AIMessage(content="Based on our conversation, you are Darshan, a data scientist working on Agentic AI. \n\nIs there anything else you'd like to tell me about yourself? I'm always happy to learn more. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 233, 'total_tokens': 283, 'completion_time': 0.090909091, 'prompt_time': 0.009079018, 'queue_time': 0.093239002, 'total_time': 0.099988109}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7992496-1245-4c70-83f3-094cf0c923ad-0', usage_metadata={'input_tokens': 233, 'output_tokens': 50, 'total_tokens': 283})

In [184]:
# Second is the different, we haven't given any information....d8
config = {"configurable": {"session_id": "d8"}}
with_message_history.invoke([HumanMessage(content="who Am I?")], config=config)

AIMessage(content="As an AI, I don't have personal experiences or memories, so I don't know who you are.\n\nTo help me understand, could you tell me a little about yourself? For example:\n\n* What are your hobbies?\n* What do you like to do in your free time?\n* What are you interested in learning more about?\n\n\nI'm here to chat and help in any way I can! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 13, 'total_tokens': 105, 'completion_time': 0.167272727, 'prompt_time': 0.001887626, 'queue_time': 0.083940956, 'total_time': 0.169160353}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-24195883-91b7-41c3-9125-38fadbbdde57-0', usage_metadata={'input_tokens': 13, 'output_tokens': 92, 'total_tokens': 105})

In [185]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}), HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}), AIMessage(content="Tha

now we have created session id d8 so see the response...

In [186]:
config = {"configurable": {"session_id": "d8"}}
with_message_history.invoke([HumanMessage(content="who Am I?")], config=config)

AIMessage(content="That's a great question! \n\nSince I don't have access to personal information about you, I can't tell you who you are.  \n\nBut I can help you explore that question!  \n\n* **What makes you feel most like yourself?** \n* **What are some things you're proud of?**\n* **What are your values and beliefs?**\n* **What do you want to be remembered for?**\n\nThinking about these things might help you discover more about who you are. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 117, 'total_tokens': 233, 'completion_time': 0.210909091, 'prompt_time': 0.005292857, 'queue_time': 0.093278594, 'total_time': 0.216201948}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-eede2b55-5071-4718-991d-9c7fd674c871-0', usage_metadata={'input_tokens': 117, 'output_tokens': 116, 'total_tokens': 233})

## Prompt Template

In [187]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

Prompt template 1

In [188]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistence, provide answers to the query in short and answer should be as accurate as possible."),
    MessagesPlaceholder(variable_name="messages")
])

In [189]:
chain = prompt|llm

In [190]:
chain.invoke({"messages": [HumanMessage(content="Hi My name is D9")]})

AIMessage(content='Hello D9! 👋  \n\nWhat can I do for you? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 38, 'total_tokens': 57, 'completion_time': 0.034545455, 'prompt_time': 0.002347354, 'queue_time': 0.10947894400000001, 'total_time': 0.036892809}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7835a251-b64b-41c5-8cb3-6e987a403a25-0', usage_metadata={'input_tokens': 38, 'output_tokens': 19, 'total_tokens': 57})

Adding chain into Runnable...

In [191]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [192]:
config = {"configurable": {"session_id": "chat1"}}
with_message_history.invoke({"messages": [HumanMessage(content="Hi My name is D9")]}, config=config)

AIMessage(content='Hello D9! 👋  How can I help you? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 38, 'total_tokens': 55, 'completion_time': 0.030909091, 'prompt_time': 0.002340775, 'queue_time': 0.109476953, 'total_time': 0.033249866}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-16e61aca-bbcc-461b-b60b-febadf43eadc-0', usage_metadata={'input_tokens': 38, 'output_tokens': 17, 'total_tokens': 55})

In [193]:
with_message_history.invoke([HumanMessage(content="Can you store my data temporarily?")], config=config)

AIMessage(content="As a text-only AI, I don't have memory of past conversations.  \n\nEvery interaction we have is fresh.  \n\nIf you need to store data, you'll need to use an external tool or service.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 70, 'total_tokens': 120, 'completion_time': 0.090909091, 'prompt_time': 0.003823581, 'queue_time': 0.110798257, 'total_time': 0.094732672}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1abe7299-1c7e-44aa-9223-e875b9c513c2-0', usage_metadata={'input_tokens': 70, 'output_tokens': 50, 'total_tokens': 120})

In [194]:
with_message_history.invoke([HumanMessage(content="who am I?")], config=config)

AIMessage(content="You are D9, a user interacting with me, an AI assistant. 🤖  \n\nIs there anything else you'd like to know or do?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 132, 'total_tokens': 167, 'completion_time': 0.063636364, 'prompt_time': 0.005729556, 'queue_time': 0.110371989, 'total_time': 0.06936592}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5b62d97-5446-483b-a3e0-c143bc654f63-0', usage_metadata={'input_tokens': 132, 'output_tokens': 35, 'total_tokens': 167})

In [195]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}), HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}), AIMessage(content="Tha

prompt template 2

In [196]:
prompt_2 = ChatPromptTemplate.from_messages([
    ("system", "You are an intelligent AI assistent. You ahve to answer the user query accurately and in the mentioned {language}"),
    MessagesPlaceholder(variable_name="messages")
])

new_chain = prompt_2|llm

In [197]:
chain.invoke({"messages": [HumanMessage(content="tell me about you")], "language": "Gujarati"})

AIMessage(content="I am Gemma, an open-weights AI assistant.\n\nI can process and generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way.\n\nMy knowledge is based on the massive dataset I was trained on, but I don't have access to real-time information or Google Search.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 36, 'total_tokens': 107, 'completion_time': 0.129090909, 'prompt_time': 0.002379025, 'queue_time': 0.084103011, 'total_time': 0.131469934}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3c73344b-44c5-41d9-b2d6-4e026e930408-0', usage_metadata={'input_tokens': 36, 'output_tokens': 71, 'total_tokens': 107})

Adding new chain into runnable

In [198]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
# The parameter input_messages_key="messages" tells the runnable which key in the input dictionary contains the chat messages to use for the session history.

In [199]:
config = {"session_id": "chat2"}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="I an D9 and I am mastering Agentic AI")], "language":"Gujarati"},
    config=config
)
repsonse.content

"Agentic AI refers to AI systems designed to act autonomously and make decisions to achieve specific goals. \n\nYou're on the right track with your training! \n\n"

In [200]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}), HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}), AIMessage(content="Tha

### Managing the Conversation History
What if our chat history will become huge???

Trimming


One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [201]:
from langchain_core.messages import SystemMessage, trim_messages

In [202]:
trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [203]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [204]:
len(messages)

11

In [205]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")| trimmer)| prompt_2 | llm
)

RunnablePassthrough is a simple runnable that applies a transformation pipeline to its input without managing additional state, while 

RunnableWithMessageHistory extends a runnable by automatically handling chat history. The latter retrieves and updates session-specific messages, ensuring that conversational context is maintained during execution.

In [206]:
len(messages)

11

In [207]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Who ice creame do I like?")],
        "language": "English"
    }
)
response.content

"As an AI, I don't know your personal preferences!  \n\nTo figure out your favorite ice cream, think about:\n\n* **What flavors do you usually go for?** Chocolate, vanilla, fruity, something more unique?\n* **What textures do you like?** Creamy, chunky, something with swirls?\n* **Are there any toppings you can't resist?** Sprinkles, nuts, whipped cream? \n\n\nLet me know if you want to brainstorm some ideas! 😊🍨 \n\n"

In [208]:
len(messages)

11

# Chat bot
Let's connect everything together...

In [209]:
trimmer=trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [210]:
# config = {"session_id": "d9"} wrong..
config = {"configurable": {"session_id": "d9"}}

In [211]:
messages = store[config["configurable"]["session_id"]].messages

In [212]:
store

{'d9': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}), HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}), AIMessage(content="Tha

In [213]:
messages

[HumanMessage(content='Hi, I am a darshan, a data scientist.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi Darshan,\n\nIt's nice to meet you!  I'm Gemma, an AI assistant.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people use data to solve problems and uncover insights.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 21, 'total_tokens': 80, 'completion_time': 0.107272727, 'prompt_time': 0.002106825, 'queue_time': 0.09164919299999999, 'total_time': 0.109379552}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-457b4b0c-bbe8-4644-a483-919d1b05bd60-0', usage_metadata={'input_tokens': 21, 'output_tokens': 59, 'total_tokens': 80}),
 HumanMessage(content='I am working on Agentic AI.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's fascinating! Agentic AI is a really 

In [214]:
len(messages)

6

In [215]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")| trimmer)| prompt | llm
)

In [216]:
len(messages)

6

In [217]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [218]:
len(messages)

6

In [219]:
config = {'session_id': 'd9'}

In [220]:
with_message_history.invoke({"messages" : [HumanMessage(content="Tell me about me?")]}, config=config)

AIMessage(content='As an AI, I have no memory of past interactions and no information about you.  To tell you about yourself, I need you to tell me something! 😊  \n\nWhat would you like to share? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 37, 'total_tokens': 84, 'completion_time': 0.085454545, 'prompt_time': 0.002351415, 'queue_time': 0.06638585300000001, 'total_time': 0.08780596}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-58dccd99-ba71-43c1-b30e-3ff9774037b6-0', usage_metadata={'input_tokens': 37, 'output_tokens': 47, 'total_tokens': 84})

In [221]:
with_message_history.invoke({"messages" : [HumanMessage(content="I an D9 and I am mastering Agentic AI")]}, config=config)

AIMessage(content="That's great!  \n\nAgentic AI focuses on creating AI systems that can act autonomously and make decisions to achieve goals.  \n\nTo master it, you'll want to delve into:\n\n* **Goal-oriented programming:** Teaching AI to understand and pursue specific objectives.\n* **Decision-making algorithms:** Exploring how AI weighs options and chooses actions.\n* **Learning and adaptation:** Enabling AI to improve its performance through experience.\n* **Robotics and control:** Applying agentic AI to physical systems.\n\n\nGood luck with your studies! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 43, 'total_tokens': 161, 'completion_time': 0.214545455, 'prompt_time': 0.002572584, 'queue_time': 0.092012728, 'total_time': 0.217118039}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cf1063ee-ba11-4fbf-bbef-fdb9b318cdb9-0', usage_metadata={'input_to

In [222]:
len(messages)

10

In [223]:
with_message_history.invoke({"messages" : [HumanMessage(content="tell me about you?")]}, config=config)

AIMessage(content='I am Gemma, an open-weights AI assistant. I can generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way. I am still under development, but I have learned to perform many kinds of tasks.  \n\nAsk me anything!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 37, 'total_tokens': 98, 'completion_time': 0.110909091, 'prompt_time': 0.002382164, 'queue_time': 0.10891774, 'total_time': 0.113291255}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-afdc089d-0968-451e-86fc-8d82f809eb69-0', usage_metadata={'input_tokens': 37, 'output_tokens': 61, 'total_tokens': 98})

In [224]:
len(messages)

12

In [225]:
with_message_history.invoke({"messages" : [HumanMessage(content="Tell me about me?")]}, config=config)

AIMessage(content='As an AI, I have no information about you. I am a new, large language model.  \n\nTo tell me about yourself, you can ask me questions or provide information. 😊  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 37, 'total_tokens': 80, 'completion_time': 0.078181818, 'prompt_time': 0.002348815, 'queue_time': 0.10897936899999999, 'total_time': 0.080530633}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0a645c27-0763-4d28-801d-b3e30efe1f4d-0', usage_metadata={'input_tokens': 37, 'output_tokens': 43, 'total_tokens': 80})

In [226]:
len(messages)

14

In [227]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

trimming = RunnablePassthrough.assign(messages = itemgetter("messages")| trimmer)


In [228]:
len(messages)

14